In [15]:
import pandas as pd

In [16]:
# Combined Satellite Catalog: Objects from the US Satellite Catalog, Objects omitted from the US catalog, Objects which failed to reach orbit
satcat_df = pd.read_csv('../data/4589_satcat.csv').drop(columns='Unnamed: 0')
psatcat_df = pd.read_csv('../data/psatcat.tsv.csv')

C:\Users\dfarr\AppData\Local\Temp\ipykernel_10556\3574278530.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  satcat_df = pd.read_csv('../data/4589_satcat.csv').drop(columns='Unnamed: 0')


In [17]:
def check_mixed_types(df):
    mixed_types_columns = {}
    for col in df.columns:
        types = df[col].map(type).unique()
        if len(types) > 1:
            mixed_types_columns[col] = types
    return mixed_types_columns

mixed_types_columns = check_mixed_types(satcat_df)
mixed_types_columns

{'Satcat': array([<class 'float'>, <class 'int'>, <class 'str'>], dtype=object)}

In [18]:
satcat_df.shape

(71292, 41)

In [19]:
satcat_df.columns

Index(['#JCAT', 'Satcat', 'Piece', 'Type', 'Name', 'PLName', 'LDate', 'Parent',
       'SDate', 'Primary', 'DDate', 'Status', 'Dest', 'Owner', 'State',
       'Manufacturer', 'Bus', 'Motor', 'Mass', 'MassFlag', 'DryMass',
       'DryFlag', 'TotMass', 'TotFlag', 'Length', 'LFlag', 'Diameter', 'DFlag',
       'Span', 'SpanFlag', 'Shape', 'ODate', 'Perigee', 'PF', 'Apogee', 'AF',
       'Inc', 'IF', 'OpOrbit', 'OQUAL', 'AltNames'],
      dtype='object')

In [20]:
satcat_df = satcat_df.rename(columns={
    '#JCAT': 'GCAT', 
    'Satcat': 'SATCAT', 
    'Piece': 'SatPiece', 
    'Type': 'SatType', 
    'Name': 'SatName', 
    'PLName': 'PayloadName', 
    'LDate': 'LaunchDate', 
    'Parent': 'SatParent',
    'SDate': 'SeparationDate', 
    'Primary': 'CentralBody', 
    'DDate': 'PhaseEndDate', 
    'Status': 'Status', 
    'Dest': 'Destination', 
    'Owner': 'Owner/Operator', 
    'State': 'OwnerState',
    'Manufacturer': 'ManufacturerOrg', 
    'Bus': 'BusType', 
    'Motor': 'MainPropulsion', 
    'Mass': 'Mass', 
    'MassFlag': 'MassFlag', 
    'DryMass':'DryMass',
    'DryFlag':'DryFlag', 
    'TotMass':'TotMass', 
    'TotFlag':'TotFlag', 
    'Length':'Length', 
    'LFlag':'LFlag', 
    'Diameter':'Diameter', 
    'DFlag':'DFlag',
    'Span':'Span', 
    'SpanFlag':'SpanFlag', 
    'Shape':'Shape', 
    'ODate':'OrbitEpochDate', 
    'Perigee':'Perigee', 
    'PF':'PF', 
    'Apogee':'Apogee', 
    'AF':'AF',
    'Inc': 'Inclination', 
    'IF': 'IF', 
    'OpOrbit': 'OrbitType', 
    'OQUAL':'OrbitQualityNote', 
    'AltNames': 'AltNames'
})

In [21]:
# Organizations Catalog:  The country of origin, owner organization, and/or manufacturer of the object of interest
orgs_df = pd.read_csv('../data/orgs.csv').drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [22]:
orgs_df.shape

(3707, 17)

In [23]:
orgs_df.columns

Index(['#Code', 'UCode', 'StateCode', 'Type', 'Class', 'TStart', 'TStop',
       'ShortName', 'Name', 'Location', 'Longitude', 'Latitude', 'Error',
       'Parent', 'ShortEName', 'EName', 'UName'],
      dtype='object')

In [25]:
orgs_df = orgs_df.rename(columns={
    '#Code': 'OrgCode', 
    'UCode': 'UnifiedOrgCode', 
    'StateCode': 'StateCode', 
    'Type': 'OrgType', 
    'Class': 'OrgClass', 
    'TStart': 'OrgPhaseStartTime', 
    'TStop': 'OrgPhaseStopTime',
    'ShortName': 'ShortOrgName', 
    'Name': 'FullOrgName', 
    'Location': 'OrgLocation', 
    'Longitude': 'OrgLongitude', 
    'Latitude': 'OrgLatitude', 
    'Error': 'LatLongError',
    'Parent': 'ParentOrgCode', 
    'ShortEName': 'OrgShortEnglishName', 
    'EName': 'OrgFullEnglishName', 
    'UName': 'OrgUnicodeName'})

In [26]:
print(orgs_df.columns)
print(satcat_df.columns)
# satcat_df['Owner/Operator'] = orgs_df['UnifiedOrgCode']
# satcat_df['OwnerState'] = orgs_df['StateCode']

Index(['OrgCode', 'UnifiedOrgCode', 'StateCode', 'OrgType', 'OrgClass',
       'OrgPhaseStartTime', 'OrgPhaseStopTime', 'ShortOrgName', 'FullOrgName',
       'OrgLocation', 'OrgLongitude', 'OrgLatitude', 'LatLongError',
       'ParentOrgCode', 'OrgShortEnglishName', 'OrgFullEnglishName',
       'OrgUnicodeName'],
      dtype='object')
Index(['GCAT', 'SATCAT', 'SatPiece', 'SatType', 'SatName', 'PayloadName',
       'LaunchDate', 'SatParent', 'SeparationDate', 'CentralBody',
       'PhaseEndDate', 'Status', 'Destination', 'Owner/Operator', 'OwnerState',
       'ManufacturerOrg', 'BusType', 'MainPropulsion', 'Mass', 'MassFlag',
       'DryMass', 'DryFlag', 'TotMass', 'TotFlag', 'Length', 'LFlag',
       'Diameter', 'DFlag', 'Span', 'SpanFlag', 'Shape', 'OrbitEpochDate',
       'Perigee', 'PF', 'Apogee', 'AF', 'Inclination', 'IF', 'OrbitType',
       'OrbitQualityNote', 'AltNames'],
      dtype='object')


In [43]:
objects_by_owner = satcat_df.groupby('Owner/Operator').size().reset_index(name='Count')
objects_by_owner = objects_by_owner[(objects_by_owner['Owner/Operator'] != '-') & (objects_by_owner['Count'] > 49)]
print(f"There are {objects_by_owner.shape[0]} owner/operators with more than 50 satellites.")

There are 112 owner/operators with more than 50 satellites.


In [45]:
satcat_df['LaunchDate'] = pd.to_datetime(satcat_df['LaunchDate'], errors='coerce')
satcat_df['OrbitEpochDate'] = pd.to_datetime(satcat_df['OrbitEpochDate'], errors='coerce')
satcat_df['PhaseEndDate'] = pd.to_datetime(satcat_df['PhaseEndDate'][satcat_df['PhaseEndDate'].notna()], errors='coerce')
satcat_df['SeparationDate'] = pd.to_datetime(satcat_df['SeparationDate'], errors='coerce')

In [47]:
satcat_df['LaunchYear'] = satcat_df['LaunchDate'].dt.year
satcat_df['LaunchMonthYear'] = satcat_df['LaunchDate'].dt.to_period('M').astype(str)
satcat_df[['LaunchDate', 'LaunchMonthYear', 'LaunchYear']].head()

,LaunchDate,LaunchMonthYear,LaunchYear
0,1957-10-04,1957-10,1957.0
1,1957-10-04,1957-10,1957.0
2,1957-11-03,1957-11,1957.0
3,1958-02-01,1958-02,1958.0
4,1958-03-17,1958-03,1958.0


In [53]:
launches_over_time = satcat_df.groupby(['LaunchMonthYear']).size().reset_index(name='LaunchCount')
launches_over_time['LaunchCountCum'] = launches_over_time['LaunchCount'].cumsum()
launches_over_time

,LaunchMonthYear,LaunchCount,LaunchCountCum
0,1957-10,4,4
1,1957-11,3,7
2,1958-02,2,9
3,1958-03,5,14
4,1958-05,5,19
...,...,...,...
782,2024-03,301,70459
783,2024-04,255,70714
784,2024-05,322,71036
785,2024-06,103,71139


In [55]:
dec_status_values = ['E', 'S', 'R', 'D', 'L', 'LF', 'AS', 'AR', 'AR IN', 'F', 'AF', 'C', 'ERR']

In [56]:
# Get a df with decayed/de-orbited objects
dec_obj_df = satcat_df[(satcat_df['Status'].isin(dec_status_values)) & (satcat_df['PhaseEndDate'].notna())]

# Get a df with active objects
act_obj_df = satcat_df[~satcat_df['Status'].isin(dec_status_values)]

In [63]:
launches_over_time['LaunchCountCum'] - dec_obj_df.shape[0]

0     -29492
1     -29489
2     -29487
3     -29482
4     -29477
       ...  
782    40963
783    41218
784    41540
785    41643
786    41796
Name: LaunchCountCum, Length: 787, dtype: int64

In [ ]:
# Get number of active satellites launched per year
all_obj_df['LaunchYear'] = all_obj_df['LDate'].dt.year
launches_per_yr = all_obj_df.groupby('LaunchYear').size()
all_obj_df[all_obj_df['LaunchYear'] == 1957]

In [ ]:
# Get number of satellites decayed each year
dec_obj_df['DecayYear'] = dec_obj_df['DDate'].dt.year
decays_per_yr = dec_obj_df.groupby('DecayYear').size()

In [ ]:
# Create a dataframe to track sats in orbit each year
years = range(int(all_obj_df['LDate'].dt.year.min()), int(all_obj_df['LDate'].dt.year.max() + 1))
satellites_in_orbit = pd.DataFrame(years, columns=['Year']).set_index('Year')

In [ ]:
launches_per_yr.cumsum()

In [ ]:
satellites_in_orbit['Launched'] = launches_per_yr.cumsum()
satellites_in_orbit['Decayed'] = decays_per_yr.cumsum()
satellites_in_orbit = satellites_in_orbit.fillna(0)
satellites_in_orbit['InOrbit'] = satellites_in_orbit['Launched'] - satellites_in_orbit['Decayed']

In [ ]:
satellites_in_orbit.sample(5)